In [ ]:
import requests
from bs4 import BeautifulSoup
from prefect import task, Flow, Parameter


@task(tags=["web"])
def retrieve_url(url):
    """
    Given a URL (string), retrieves html and
    returns the html as a string.
    """

    html = requests.get(url)
    if html.ok:
        return html.text
    else:
        raise ValueError("{} could not be retrieved.".format(url))

In [ ]:
@task
def scrape_dialogue(episode_html):
    """
    Given a string of html representing an episode page,
    returns a tuple of (title, [(character, text)]) of the dialogue from that
    episode
    """

    episode = BeautifulSoup(episode_html, 'html.parser')
    
    title = episode.title.text.rstrip(' *').replace("'", "''")
    convos = episode.find_all('b') or episode.find_all('span', {'class': 'char'})
    dialogue = []
    for item in convos:
        who = item.text.rstrip(': ').rstrip(' *').replace("'", "''")
        what = str(item.next_sibling).rstrip(' *').replace("'", "''")
        dialogue.append((who, what))
    return (title, dialogue)

In [ ]:
with Flow("xfiles") as flow:
    url = Parameter("url")
    episode = retrieve_url(url)
    dialogue = scrape_dialogue(episode)


flow.visualize()

In [ ]:
episode_url = "http://www.insidethex.co.uk/transcrp/scrp320.htm"
outer_space = flow.run(parameters={"url": episode_url},
                       return_tasks=[dialogue])

state = outer_space.result[dialogue] # the `State` object for the dialogue task
first_five_spoken_lines = state.result[1][:5] # state.result is a tuple (episode_name, [dialogue])
print(''.join([f'{speaker}: {words}' for speaker, words in first_five_spoken_lines]))

In [ ]:
@task
def create_episode_list(base_url, main_html, bypass):
    """
    Given the main page html, creates a list of episode URLs
    """

    if bypass:
        return [base_url]
    
    main_page = BeautifulSoup(main_html, 'html.parser')
    
    episodes = []
    for link in main_page.find_all('a'):
        url = link.get('href')
        if 'transcrp/scrp' in (url or ''):
            episodes.append(base_url + url)

    return episodes

In [ ]:
with Flow("xfiles") as flow:
    url = Parameter("url")
    bypass = Parameter("bypass", default=False, required=False)
    home_page = retrieve_url(url)
    episodes = create_episode_list(url, home_page, bypass=bypass)
    episode = retrieve_url.map(episodes)
    dialogue = scrape_dialogue.map(episode)

In [ ]:
episode_url = "http://www.insidethex.co.uk/transcrp/scrp320.htm"
outer_space = flow.run(parameters={"url": episode_url, "bypass": True},
                       start_tasks=[bypass, episodes, url],
                       return_tasks=[dialogue])

In [ ]:
flow.visualize()

In [ ]:
%%time
scraped_state = flow.run(parameters={"url": "http://www.insidethex.co.uk/"}, return_tasks=[dialogue])
#    CPU times: user 7.48 s, sys: 241 ms, total: 7.73 s
#    Wall time: 4min 46s

dialogue_state = scraped_state.result[dialogue] # list of State objects
print('\n'.join([f'{s.result[0]}: {s}' for s in dialogue_state[:5]]))

In [ ]:
from prefect.engine.executors import DaskExecutor

executor = DaskExecutor(local_processes=True)

In [ ]:
%%time
scraped_state = flow.run(parameters={"url": "http://www.insidethex.co.uk/"},
               return_tasks=flow.tasks,
               executor=executor)

#    CPU times: user 9.7 s, sys: 1.67 s, total: 11.4 s
#    Wall time: 1min 34s

dialogue_state = scraped_state.result[dialogue] # list of State objects
print('\n'.join([f'{s.result[0]}: {s}' for s in dialogue_state[:5]]))

In [ ]:
from prefect.tasks.database import SQLiteScriptTask


create_db = SQLiteScriptTask(name="Create DB",
                             db="xfiles_db.sqlite",
                             script="CREATE TABLE IF NOT EXISTS XFILES (EPISODE TEXT, CHARACTER TEXT, TEXT TEXT)",
                             tags=["db"])


@task
def create_episode_script(episode):
    title, dialogue = episode
    insert_cmd = "INSERT INTO XFILES (EPISODE, CHARACTER, TEXT) VALUES\n"
    values = ',\n'.join(["('{0}', '{1}', '{2}')".format(title, *row) for row in dialogue]) + ";"
    return insert_cmd + values


insert_episode = SQLiteScriptTask(name="Insert Episode",
                                  db="xfiles_db.sqlite", 
                                  tags=["db"])

In [ ]:
from prefect import unmapped

with flow:
    db = create_db()
    ep_script = create_episode_script.map(episode=dialogue)
    final = insert_episode.map(ep_script, upstream_tasks=[unmapped(db)])

In [ ]:
flow.visualize()

In [ ]:
state = flow.run(parameters={"url": "http://www.insidethex.co.uk/"},
                 executor=executor,
                 task_states=scraped_state.result,
                 start_tasks=[ep_script, db])

In [ ]:
import sqlite3
from contextlib import closing


with closing(sqlite3.connect("xfiles_db.sqlite")) as conn:
        with closing(conn.cursor()) as c:
            create_cmd = '''SELECT * FROM XFILES WHERE TEXT LIKE '%programming%';'''
            c.execute(create_cmd)
            programming_mentions = c.fetchall()

print(programming_mentions)

In [ ]:
final = flow.run(parameters={"url": "http://www.insidethex.co.uk/transcrp/tlg105.htm",
                             "bypass": True},
                 start_tasks=[bypass, db, episodes, url])


with closing(sqlite3.connect("xfiles_db.sqlite")) as conn:
        with closing(conn.cursor()) as c:
            create_cmd = '''SELECT * FROM XFILES WHERE TEXT LIKE '%programming%';'''
            c.execute(create_cmd)
            programming_mentions = c.fetchall()

print(programming_mentions)